In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras_tuner.tuners import RandomSearch



In [2]:
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],)))
    
    for i in range(hp.Int('num_layers', 2, 10)):
        model.add(layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=256, step=32),
                               activation='relu'))
    
    model.add(layers.Dense(units=len(set(y)), activation='softmax'))
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.01,0.0001])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [5]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='my_dir',
    project_name='iris_tune'
)


Reloading Tuner from my_dir\iris_tune\tuner0.json


In [6]:
tuner.search_space_summary()

Search space summary
Default search space size: 12
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 10, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.01, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
units_

In [7]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))


In [8]:
tuner.results_summary()

Results summary
Results in my_dir\iris_tune
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layers: 8
units_0: 96
units_1: 224
learning_rate: 0.001
units_2: 64
units_3: 128
units_4: 224
units_5: 224
units_6: 128
units_7: 32
Score: 0.9666666388511658

Trial 3 summary
Hyperparameters:
num_layers: 3
units_0: 96
units_1: 32
learning_rate: 0.001
units_2: 256
units_3: 96
units_4: 160
units_5: 64
units_6: 128
units_7: 64
Score: 0.9666666388511658

Trial 0 summary
Hyperparameters:
num_layers: 4
units_0: 224
units_1: 128
learning_rate: 0.001
units_2: 32
units_3: 32
Score: 0.9666666388511658

Trial 1 summary
Hyperparameters:
num_layers: 7
units_0: 32
units_1: 32
learning_rate: 0.001
units_2: 224
units_3: 224
units_4: 32
units_5: 32
units_6: 32
Score: 0.8999999761581421

Trial 4 summary
Hyperparameters:
num_layers: 10
units_0: 256
units_1: 128
learning_rate: 0.0001
units_2: 32
units_3: 96
units_4: 64
units_5: 32
units_6: 32
units_7: 64


In [9]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 96)                480       
                                                                 
 dense_1 (Dense)             (None, 224)               21728     
                                                                 
 dense_2 (Dense)             (None, 64)                14400     
                                                                 
 dense_3 (Dense)             (None, 128)               8320      
                                                                 
 dense_4 (Dense)             (None, 224)               28896     
                                                                 
 dense_5 (Dense)             (None, 224)               50400     
                                                                 
 dense_6 (Dense)             (None, 128)               2